In [13]:
from typing import List

from qiskit import QuantumCircuit, transpile
from qiskit.circuit.random import random_circuit

from quantum_serverless import QuantumServerless, remote, get, put

In [4]:
serverless = QuantumServerless()

serverless

QuantumServerless:
 | selected cluster: <Cluster: Local (all cores)>

In [5]:
circuit = random_circuit(5, 3)

circuit.draw()

┌───┐             
q_0: ──■────────────┤ X ├──────■──────
     ┌─┴─┐          └─┬─┘    ┌─┴─┐    
q_1: ┤ X ├────────────■──────┤ X ├────
     └─┬─┘┌───┐       │      └─┬─┘    
q_2: ──┼──┤ I ├──■────┼────────■──────
       │  └───┘┌─┴─┐  │      ┌───┐    
q_3: ──■───────┤ Y ├──┼──────┤ T ├────
     ┌───┐     └───┘  │  ┌───┴───┴───┐
q_4: ┤ Z ├────────────■──┤ Rx(1.041) ├
     └───┘               └───────────┘

In [10]:
# object

with serverless.context():
    # make any object as remote object
    circuit_reference = put(circuit)

    print("Circuit reference:", circuit_reference)
    
    # to get any object from remote to local call `get` function
    retrieved_object = get(circuit_reference)
    print("Retrieved object", retrieved_object)

Circuit reference: ObjectRef(00ffffffffffffffffffffffffffffffffffffff0100000004000000)
Retrieved object                     ┌───┐             
q_0: ──■────────────┤ X ├──────■──────
     ┌─┴─┐          └─┬─┘    ┌─┴─┐    
q_1: ┤ X ├────────────■──────┤ X ├────
     └─┬─┘┌───┐       │      └─┬─┘    
q_2: ──┼──┤ I ├──■────┼────────■──────
       │  └───┘┌─┴─┐  │      ┌───┐    
q_3: ──■───────┤ Y ├──┼──────┤ T ├────
     ┌───┐     └───┘  │  ┌───┴───┴───┐
q_4: ┤ Z ├────────────■──┤ Rx(1.041) ├
     └───┘               └───────────┘


In [15]:
circuit_batches = [
    [random_circuit(5, 3) for _ in range(2)]
    for n_batch in range(3)
]
circuit_batches

[[<qiskit.circuit.quantumcircuit.QuantumCircuit at 0x7fd6d8b478e0>,

In [17]:
# task

# in order to make any function a remote/parallel function annotate it with `remote` decorator

@remote
def remote_transpile(circuits: List[QuantumCircuit]):
    return transpile(circuits)

with serverless.context():
    # to call this function by arring `remote` to function call `<function_name>.remote`
    remote_function_reference = remote_transpile.remote(circuit_batches[0])
    print("Function reference:", remote_function_reference)
    
    # to get function results we again using `get`
    result = get(remote_function_reference)
    print("Function result:", result)

Function reference: ObjectRef(16310a0f0a45af5cffffffffffffffffffffffff0100000001000000)
Function result: [<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fd6b8f2d420>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fd729394c40>]


In [18]:
# we can run multiple remote function in parallel

with serverless.context():
    # to call this function by arring `remote` to function call `<function_name>.remote`
    remote_function_references = [
        remote_transpile.remote(circuit_batch)
        for circuit_batch in circuit_batches
    ]
    print("Functions references:", remote_function_references)
    
    # to get function results we again using `get`
    result = get(remote_function_references)
    print("Functions resulst:", result)

Functions references: [ObjectRef(c2668a65bda616c1ffffffffffffffffffffffff0100000001000000), ObjectRef(32d950ec0ccf9d2affffffffffffffffffffffff0100000001000000), ObjectRef(e0dc174c83599034ffffffffffffffffffffffff0100000001000000)]
Functions resulst: [[<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fd7187b6ef0>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fd729220d60>], [<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fd729221810>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fd729220280>], [<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fd7292218a0>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fd72901eb90>]]
